### Seq2seq

In [5]:
import numpy as np

In [6]:
import numpy as np
import csv
# import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

import numpy as np
import csv
# import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils

In [7]:
# load dataset
with open('../../data/food_review.csv') as f:
    reader = csv.reader(f)
    l = list(reader)

In [8]:
n_samples =10000

In [16]:
# encoder and decoder
# Appending SOS andEOS to target data (decoder)
enc = [l[i][0] for i in range(1, len(l))] [0:n_samples]
dec = ['SOS_ '+ l[i][1] + ' _EOS' for i in range(1, len(l))][0:n_samples]
enc_dec = enc + dec

num_enc_samples = len(enc)
num_dec_samples = len(dec)

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils

tokenizer = Tokenizer() 
tokenizer.fit_on_texts(enc_dec) 
vocab_size = len(tokenizer.word_index) + 1

# encoder source data
source_token = tokenizer.texts_to_sequences(enc)
max_encoder_seq_length = max([len(sentence) for sentence in source_token])
source_padded = pad_sequences(source_token, maxlen=max_encoder_seq_length, padding = "post")
# decoder target data
target_token = tokenizer.texts_to_sequences(dec)
max_decoder_seq_length = max([len(sentence) for sentence in target_token])
target_padded = pad_sequences(target_token, maxlen=max_decoder_seq_length, padding = "post")

In [17]:
print('num_en_samples: ', num_enc_samples)
print('num_de_samples: ', num_dec_samples)
print('vocab_size: ',vocab_size)
print('max_encoder_seq_length: ', max_encoder_seq_length)
print('max_decoder_seq_length: ', max_decoder_seq_length)

num_en_samples:  10000
num_de_samples:  10000
vocab_size:  22435
max_encoder_seq_length:  501
max_decoder_seq_length:  28


In [21]:
# running time calculation
import timeit
start = timeit.default_timer()

n_samples = num_enc_samples #100

# prepare data for the LSTM
decoder_input_data, decoder_target_data = list(), list()
for i in range(n_samples):
    dec_input = target_padded[i][:-1]
    target = target_padded[i][1:]     
    decoder_input_data.append(dec_input) #tar2_encoded
    decoder_target_data.append(target)

encoder_input_data = source_padded #np.array(X1)
decoder_input_data = np.array(decoder_input_data)
decoder_target_data = np.array(decoder_target_data)

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

(10000, 501)
(10000, 27)
(10000, 27)
Time: 0.02 s


### Embedding from here

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed,Dropout

# Defining some constants: 
vec_len       = 50#300   # Length of the vector that we willl get from the embedding layer
latent_dim    = 32#1024  # Hidden layers dimension 
dropout_rate  = 0.2   # Rate of the dropout layers

# Define an input sequence and process it.
# Input layer of the encoder :
encoder_input = Input(shape=(None,))

# Hidden layers of the encoder :
encoder_embedding = Embedding(input_dim = vocab_size, output_dim = vec_len)(encoder_input)
encoder_dropout   = (TimeDistributed(Dropout(rate = dropout_rate)))(encoder_embedding)
encoder_LSTM      = CuDNNLSTM(latent_dim, return_sequences=True)(encoder_dropout)

# Output layer of the encoder :
encoder_LSTM2_layer = CuDNNLSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM2_layer(encoder_LSTM)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
# Input layer of the decoder :
decoder_input = Input(shape=(None,))

# Hidden layers of the decoder :
decoder_embedding_layer = Embedding(input_dim = vocab_size, output_dim = vec_len)
decoder_embedding = decoder_embedding_layer(decoder_input)

decoder_dropout_layer = (TimeDistributed(Dropout(rate = dropout_rate)))
decoder_dropout = decoder_dropout_layer(decoder_embedding)

decoder_LSTM_layer = CuDNNLSTM(latent_dim, return_sequences=True)
decoder_LSTM = decoder_LSTM_layer(decoder_dropout, initial_state = encoder_states)

decoder_LSTM_2_layer = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
decoder_LSTM_2,_,_ = decoder_LSTM_2_layer(decoder_LSTM)

# Output layer of the decoder :
decoder_dense = Dense(vocab_size, activation='linear', name='decoder_output')
decoder_outputs = decoder_dense(decoder_LSTM_2)


# Define encoder model
encoder_model = Model(encoder_input, encoder_states)

# Define training model
model = Model([encoder_input, decoder_input], decoder_outputs)

# Define decoder model
dec_h = Input(shape=(latent_dim,))
dec_c = Input(shape=(latent_dim,))
dec_states_inputs = [dec_h, dec_c]
decoder_outputs, state_h, state_c = decoder_LSTM_2_layer(decoder_embedding, initial_state=dec_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_input] + dec_states_inputs, [decoder_outputs] + decoder_states)

model.summary() 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 50)     1121750     input_17[0][0]                   
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
time_distributed_9 (TimeDistrib (None, None, 50)     0           embedding_9[0][0]                
__________________________________________________________________________________________________
embedding_

In [32]:
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline

learning_rate=0.01    
batch_size    = 32#64    # Batch size
epochs        = 1#30    # Number of epochs

# define loss function: use sparse_softmax_cross_entropy_with_logits
def sparse_loss(targets, decoder_outputs):
    return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=targets, logits=decoder_outputs)

# Define a checkpoint callback :
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

# Run training
decoder_target = tf.placeholder(dtype='int32', shape=(None, vocab_size))   

model.compile(optimizer=Adam(lr=learning_rate),
                loss=sparse_loss,
                target_tensors=[decoder_target])

model.fit([encoder_input_data,
               decoder_input_data],
               decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.08,
          callbacks = callbacks_list)


Train on 9200 samples, validate on 800 samples
9184/9200 [============================>.] - ETA: 0s - loss: 1.7493
Epoch 00001: val_loss improved from inf to 1.21521, saving model to Weights-001--1.21521.hdf5
9200/9200 [==============================] - 53s 6ms/sample - loss: 1.7485 - val_loss: 1.2152


In [ ]:
# # Use sklearn utility to convert label strings to numbered index
# encoder = LabelEncoder()
# encoder.fit(train_label)
# y_train = encoder.transform(train_label)
# y_test = encoder.transform(test_label)

# # Converts the labels to a one-hot representation
# num_classes = np.max(y_train) + 1
# y_train = utils.to_categorical(y_train, num_classes)
# y_test = utils.to_categorical(y_test, num_classes)